In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.models import vgg19, VGG19_Weights

In [ ]:
model = vgg19(VGG19_Weights.DEFAULT)
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:04<00:00, 135MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
class UNET(nn.Module):
    def __init__(self, encoder, center, decoder):
        super().__init__()
        self.encoder = encoder
        self.center = center
        self.decoder = decoder

    def forward(self, x):
        encoder_outputs = self.encoder(x)
        center_output = self.center(encoder_outputs[-1])
        decoder_output = self.decoder(center_output, encoder_outputs)

        return decoder_output

In [ ]:
class Encoder(nn.Module):
    def __init__(self, pretrained_network):
        super().__init__()
        self.encoder = pretrained_network

    def forward(self, x):
        encoder_outputs = []

        for layers in self.encoder.features:
            x = layers(x)
            encoder_outputs.append(x)

        return encoder_outputs

In [ ]:
class Center(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = (3,3), padding = 1)
        self.bn1 = nn.BatchNorm2d(1024)
        self.conv2 = nn.Conv2d(in_channels = 1024, out_channels = 1024, kernel_size = (3,3), padding = 1)
        self.bn2 = nn.BatchNorm2d(1024)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv5_up = nn.Conv2d(in_channels = 1024, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.conv5_1 = nn.Conv2d(in_channels = 1024, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn5_1 = nn.BatchNorm2d(512)
        self.conv5_2 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn5_2 = nn.BatchNorm2d(512)
        self.conv5_3 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn5_3 = nn.BatchNorm2d(512)
        self.conv5_4 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn5_4 = nn.BatchNorm2d(512)

        self.conv4_up = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.conv4_1 = nn.Conv2d(in_channels = 1024, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn4_1 = nn.BatchNorm2d(512)
        self.conv4_2 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn4_2 = nn.BatchNorm2d(512)
        self.conv4_3 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn4_3 = nn.BatchNorm2d(512)
        self.conv4_4 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3,3), padding = 1)
        self.bn4_4 = nn.BatchNorm2d(512)

        self.conv3_up = nn.Conv2d(in_channels = 512, out_channels = 256, kernel_size = (3,3), padding = 1)
        self.conv3_1 = nn.Conv2d(in_channels = 512, out_channels = 256, kernel_size = (3,3), padding = 1)
        self.bn3_1 = nn.BatchNorm2d(256)
        self.conv3_2 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = (3,3), padding = 1)
        self.bn3_2 = nn.BatchNorm2d(256)
        self.conv3_3 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = (3,3), padding = 1)
        self.bn3_3 = nn.BatchNorm2d(256)
        self.conv3_4 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = (3,3), padding = 1)
        self.bn3_4 = nn.BatchNorm2d(256)

        self.conv2_up = nn.Conv2d(in_channels = 256, out_channels = 128, kernel_size = (3,3), padding = 1)
        self.conv2_1 = nn.Conv2d(in_channels = 256, out_channels = 128, kernel_size = (3,3), padding = 1)
        self.bn2_1 = nn.BatchNorm2d(128)
        self.conv2_2 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = (3,3), padding = 1)
        self.bn2_2 = nn.BatchNorm2d(128)

        self.conv1_up = nn.Conv2d(in_channels = 128, out_channels = 64, kernel_size = (3,3), padding = 1)
        self.conv1_1 = nn.Conv2d(in_channels = 128, out_channels = 64, kernel_size = (3,3), padding = 1)
        self.bn1_1 = nn.BatchNorm2d(64)
        self.conv1_2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 1)
        self.bn1_2 = nn.BatchNorm2d(64)

        self.conv_final = nn.Conv2d(in_channels = 64, out_channels = 2, kernel_size = (1,1))

    def forward(self, x, encoder_outputs):
        x = F.interpolate(x, scale_factor = 2, mode = 'nearest')
        x = self.conv5_up(x)
        x = self.relu(x)
        x = torch.cat((x, encoder_outputs[51]), dim = 1)
        x = self.conv5_1(x)
        x = self.bn5_1(x)
        x = self.relu(x)
        x = self.conv5_2(x)
        x = self.bn5_2(x)
        x = self.relu(x)
        x = self.conv5_3(x)
        x = self.bn5_3(x)
        x = self.relu(x)
        x = self.conv5_4(x)
        x = self.bn5_4(x)
        x = self.relu(x)

        x = F.interpolate(x, scale_factor = 2, mode = 'nearest')
        x = self.conv4_up(x)
        x = self.relu(x)
        x = torch.cat((x, encoder_outputs[38]), dim = 1)
        x = self.conv4_1(x)
        x = self.bn4_1(x)
        x = self.relu(x)
        x = self.conv4_2(x)
        x = self.bn4_2(x)
        x = self.relu(x)
        x = self.conv4_3(x)
        x = self.bn4_3(x)
        x = self.relu(x)
        x = self.conv4_4(x)
        x = self.bn4_4(x)
        x = self.relu(x)

        x = F.interpolate(x, scale_factor = 2, mode = 'nearest')
        x = self.conv3_up(x)
        x = self.relu(x)
        x = torch.cat((x, encoder_outputs[25]), dim = 1)
        x = self.conv3_1(x)
        x = self.bn3_1(x)
        x = self.relu(x)
        x = self.conv3_2(x)
        x = self.bn3_2(x)
        x = self.relu(x)
        x = self.conv3_3(x)
        x = self.bn3_3(x)
        x = self.relu(x)
        x = self.conv3_4(x)
        x = self.bn3_4(x)
        x = self.relu(x)

        x = F.interpolate(x, scale_factor = 2, mode = 'nearest')
        x = self.conv2_up(x)
        x = self.relu(x)
        x = torch.cat((x, encoder_outputs[12]), dim = 1)
        x = self.conv2_1(x)
        x = self.bn2_1(x)
        x = self.relu(x)
        x = self.conv2_2(x)
        x = self.bn2_2(x)
        x = self.relu(x)

        x = F.interpolate(x, scale_factor = 2, mode = 'nearest')
        x = self.conv1_up(x)
        x = self.relu(x)
        x = torch.cat((x, encoder_outputs[5]), dim = 1)
        x = self.conv1_1(x)
        x = self.bn1_1(x)
        x = self.relu(x)
        x = self.conv1_2(x)
        x = self.bn1_2(x)
        x = self.relu(x)

        x = self.conv_final(x)

        return x

In [ ]:
import torch
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0)

In [ ]:
img = Image.open('/kaggle/input/sai-vessel-segmentation2/all/train/34_training.tif')
mask = Image.open('/kaggle/input/sai-vessel-segmentation2/all/train/34_manual1.gif')

print("Image Type:", type(img))
print("Mask Type:", type(mask))
print(f"Image Shape: {np.array(img).shape} | Mask Sahpe: {np.array(mask).shape}")

print("Unique values in Mask:",np.unique(mask))
print("Image Data Type:", np.array(img).dtype)
print("Mask Data Type:", np.array(mask).dtype)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.title("Original Image")
plt.imshow(img)
plt.subplot(1,2,2)
plt.title("Mask")
plt.imshow(mask)
plt.show()

In [ ]:
def reshape(path):
    original_image = Image.open(path)
    reshaped_image = original_image.resize((512,512), PIL.Image.NEAREST)
    return reshaped_image

In [ ]:
img = reshape('/kaggle/input/sai-vessel-segmentation2/all/train/34_training.tif')
mask = reshape('/kaggle/input/sai-vessel-segmentation2/all/train/34_manual1.gif')

print("Image Type:", type(img))
print("Mask Type:", type(mask))
print(f"Image Shape: {np.array(img).shape} | Mask Sahpe: {np.array(mask).shape}")

print("Unique values in Mask:",np.unique(mask))
print("Image Data Type:", np.array(img).dtype)
print("Mask Data Type:", np.array(mask).dtype)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.title("Original Image")
plt.imshow(img)
plt.subplot(1,2,2)
plt.title("Mask")
plt.imshow(mask)
plt.show()

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, path, transform = None):
        super().__init__()
        self.path = path
        _,_,self.files = next(os.walk(path))
        self.length = int(len(self.files)/2) - 4
        self.transform = Compose([ToTensor(), Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        idx = idx + 21
        path = self.path + str(idx) + "_training.tif"
        img = reshape(path)
        img = self.transform(img)

        path = self.path + str(idx) + "_manual1.gif"
        mask = reshape(path)
        mask = np.array(mask)
        mask = torch.from_numpy(mask).type(torch.long)
        mask[mask == 255] = 1

        return img,mask

class ValidationDataset(Dataset):
    def __init__(self, path, transform = None):
        super().__init__()
        self.path = path
        _,_,self.files = next(os.walk(path))
        self.length = int(len(self.files)/2) - 16
        self.transform = Compose([ToTensor(), Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        idx = idx + 37
        path = self.path + str(idx) + "_training.tif"
        img = reshape(path)
        img = self.transform(img)

        path = self.path + str(idx) + "_manual1.gif"
        mask = reshape(path)
        mask = np.array(mask)
        mask = torch.from_numpy(mask).type(torch.long)
        mask[mask == 255] = 1

        return img,mask

In [ ]:
train_dataset = TrainDataset("/kaggle/input/sai-vessel-segmentation2/all/train/")
validation_dataset = ValidationDataset("/kaggle/input/sai-vessel-segmentation2/all/train/")

batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size)

In [ ]:
img,mask=train_dataset[13]

print("Image Shape:",img.shape,"Mask Shape:",mask.shape,"Image dtype:",img.dtype, "Mask dtype:", mask.dtype)
print("Unique values in Mask:",np.unique(mask))

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.title("Original Image 512 x 512 from Train_Dataset")
plt.imshow(torch.permute(img,(1,2,0)))

plt.subplot(1,2,2)
plt.title("Mask 512 x 512 from Train_Dataset")
plt.imshow(mask)
plt.show()

In [ ]:
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    track_loss = 0
    XintY = 0
    X = 0
    Y = 0
    for i, (imgs, masks) in enumerate(dataloader):
        imgs = imgs.to(device)
        masks = masks.to(device)

        preds = model(imgs)

        loss = loss_fn(preds,masks)

        track_loss += loss.item()

        predclass = torch.argmax(preds,dim=1)

        Y += predclass.sum().item()
        X += masks.sum().item()


        predclass[predclass==0] = 2

        XintY += (predclass==masks).type(torch.float).sum().item()

        print("Trainig Batch",i+1,":","2*XintY:",2*XintY,"X:",X,"Y:",Y, "X+Y:",X+Y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss = round(track_loss/(i+1),2)
        running_dice_coef = round(((2*XintY)/(X+Y)),2)

        print("Training Batch", i+1,":","/",len(dataloader), "Running Loss:",running_loss, "Running Dice_Coef:",running_dice_coef)

    epoch_loss = running_loss
    epoch_dice_coef = running_dice_coef
    return epoch_loss, epoch_dice_coef


def validation_one_epoch(dataloader, model,loss_fn):
    model.eval()
    track_loss = 0
    XintY = 0
    X = 0
    Y = 0
    with torch.no_grad():
        for i, (imgs, masks) in enumerate(dataloader):
            imgs = imgs.to(device)
            masks = masks.to(device)

            preds = model(imgs)

            loss = loss_fn(preds,masks)

            track_loss += loss.item()

            predclass = torch.argmax(preds,dim=1)

            Y += predclass.sum().item()
            X += masks.sum().item()

            predclass[predclass==0] = 2

            XintY += (predclass==masks).type(torch.float).sum().item()

            print("Validation Batch",i+1,":","2*XintY:",2*XintY,"X:",X,"Y:",Y, "X+Y:",X+Y)


            running_loss = round(track_loss/(i+1),2)
            running_dice_coef = round(((2*XintY)/(X+Y)),2)

            print("Validation Batch", i+1,":","/",len(dataloader), "Running Loss:",running_loss, "Running Dice_Coef:",running_dice_coef)

    epoch_loss = running_loss
    epoch_dice_coef = running_dice_coef
    return epoch_loss, epoch_dice_coef

In [ ]:
pretrained_network = vgg19_bn(weights = VGG19_BN_Weights.DEFAULT)

for param in pretrained_network.features.parameters():
    param.requires_grad = False

encoder = Encoder(pretrained_network).to(device)
center = Center().to(device)
decoder = Decoder().to(device)

model = UNET(encoder,center, decoder).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.001)
epochs = 60

for i in range(epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_dice_coef=train_one_epoch(train_dataloader,model,loss_fn,optimizer)
    print("Training Epoch Loss:", train_epoch_loss, "Training Epoch Dice_Coef:", train_epoch_dice_coef)
    val_epoch_loss, val_epoch_dice_coef = validation_one_epoch(validation_dataloader,model,loss_fn)
    print("Validation Epoch Loss:", val_epoch_loss, "Validation Epoch Dice_Coef:", val_epoch_dice_coef)
    print("--------------------------------------------------")


for param in pretrained_network.features.parameters():
    param.requires_grad=True

for i in range(epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_dice_coef=train_one_epoch(train_dataloader,model,loss_fn,optimizer)
    print("Training Epoch Loss:", train_epoch_loss, "Training Epoch Dice_Coef:", train_epoch_dice_coef)
    val_epoch_loss, val_epoch_dice_coef=validation_one_epoch(validation_dataloader,model,loss_fn)
    print("Validation Epoch Loss:", val_epoch_loss, "Validation Epoch Dice_Coef:", val_epoch_dice_coef)
    print("--------------------------------------------------")

In [ ]:
class FinalTrainDataset(Dataset):
    def __init__(self, path, transform = None):
        super().__init__()
        self.path = path
        _,_,self.files = next(os.walk(path))
        self.length = int(len(self.files)/2)
        self.transform = Compose([ToTensor(), Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        idx = idx + 21
        path = self.path + str(idx) + "_training.tif"
        img = reshape(path)
        img = self.transform(img)

        path = self.path + str(idx) + "_manual1.gif"
        mask = reshape(path)
        mask = np.array(mask)
        mask = torch.from_numpy(mask).type(torch.long)
        mask[mask == 255] = 1

        return img,mask

In [ ]:
train_dataset = FinalTrainDataset("/kaggle/input/sai-vessel-segmentation2/all/train/")
train_dataloader = DataLoader(train_dataset, batch_size, True)

for i in range(epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_dice_coef=train_one_epoch(train_dataloader,model,loss_fn,optimizer)
    print("Training Epoch Loss:", train_epoch_loss, "Training Epoch Dice_Coef:", train_epoch_dice_coef)
    print("--------------------------------------------------")

In [ ]:
class TestDataset(Dataset):
    def __init__(self, path, transform = None):
        super().__init__()
        self.path = path
        _,_,self.files = next(os.walk(path))
        self.length = int(len(self.files))
        self.transform = Compose([ToTensor(), Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        idx = idx + 1
        if idx <= 9:
            path = self.path + "0" + str(idx) + "_test.tif"
        else:
            path = self.path + str(idx) + "_test.tif"

        img = reshape(path)
        img = self.transform(img)

        return img

In [ ]:
test_dataset = TestDataset("/kaggle/input/sai-vessel-segmentation2/all/test/")
test_dataloader = DataLoader(test_dataset, batch_size = 2)

In [ ]:
def eval_one_epoch(dataloader, model):
    model.eval()
    outputs = []
    for i, imgs in enumerate(dataloader):
        imgs = imgs.to(device)
        preds = model(imgs)

        with torch.no_grad():
            for i in range(preds.shape[0]):
                pred = preds[i,:,:,:]
                pred = torch.argmax(pred,dim = 0)
                predf = pred.flatten()
                pixelidx = np.where(predf==1)[0]+1

                run_lengths = []

                for pxid in pixelidx:
                    if len(run_lengths) == 0:
                        run_lengths.extend((pxid,1))
                    elif pxid > prev+1:
                        run_lengths.extend((pxid,1))
                    else:
                        run_lengths[-1] += 1
                    prev=pxid

                output = ' '.join([str(r) for r in run_lengths])

                outputs.append(output)
    return outputs

outputs = eval_one_epoch(test_dataloader,model)
df = pd.DataFrame(columns=['Id','Predicted'])
df['Id'] = [str(i) for i in range(20)]
df['Predicted'] = outputs
df.to_csv("submission.csv", index=None)
df